In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.modeling.models import Sersic2D
from astropy.convolution import convolve
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/nasekhian/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/xm/sl735y1n4m30kfvcxl7h4p6w0000gn/T/ipykernel_39136/1673434397.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the coordinates from the CSV file
coordinates = pd.read_csv('coordinates2.csv')

In [3]:
# Set a fixed seed for reproducibility
np.random.seed(42)

# Define the number of galaxies to simulate
num_galaxies = 120

# Define lists to store input parameters
sersic_index_list = []
effective_radius_list = []
amplitude_list = []
ellipticity_list = []
theta_list = []
total_flux_list = []
magnitude_list = []

In [5]:
# Loop over each row in the DataFrame
for index, row in coordinates.iterrows():
    # Define the properties of the simulated galaxy for this iteration
    sersic_index = np.random.uniform(1, 2.5)  # Random Sersic index between 1 and 4
    effective_radius = np.random.uniform(3, 5)  # Random effective radius between 1 and 10
    amplitude = np.random.uniform(0.001, 0.01)  # Random amplitude between 0.1 and 1
    ellip = np.random.uniform(0.3, 0.5)  # Random ellipticity between 0 and 0.5
    theta = np.radians(np.random.uniform(0, 180))  # Random angle between 0 and 180 degrees

    # Specify the input file names
    galaxy_file = 'sci_f444w_22450.fits'
    psf_file = 'ePSFf444w.fits'

    # Load data
    galaxy_data, galaxy_header = fits.getdata(galaxy_file, header=True)
    psf_data = fits.getdata(psf_file)
    y, x = np.mgrid[:galaxy_data.shape[0], :galaxy_data.shape[1]]

    # Create a Sersic model for the simulated galaxy
    galaxy_model = Sersic2D(amplitude=amplitude, r_eff=effective_radius, n=sersic_index,
                             x_0=row['x0'], y_0=row['y0'], ellip=ellip, theta=theta)
    simulated_galaxy = galaxy_model(x, y)
    convolved_simulated = convolve(simulated_galaxy, psf_data, boundary='extend')

    # Calculate total flux
    total_flux = np.sum(simulated_galaxy)

    # Calculate magnitude
    magnitude = -2.5 * np.log10(total_flux) + 23.9

    # Append the input parameters to the lists
    sersic_index_list.append(sersic_index)
    effective_radius_list.append(effective_radius)
    amplitude_list.append(amplitude)
    ellipticity_list.append(ellip)
    theta_list.append(theta)
    total_flux_list.append(total_flux)
    magnitude_list.append(magnitude)

    # Save the convolved simulated galaxy fits file in the relevant folder
    simulation_dir = f'simulation_{index}'
    if not os.path.exists(simulation_dir):
        os.makedirs(simulation_dir)
    convolved_output_path = os.path.join(simulation_dir, f'simulated_galaxy_{index}.fits')
    fits.writeto(convolved_output_path, convolved_simulated, header=galaxy_header, overwrite=True)

    # Save the original simulated galaxy fits file in the same folder
    #simulated_output_path = os.path.join(simulation_dir, f'original_simulated_galaxy_{index}.fits')
    #fits.writeto(simulated_output_path, simulated_galaxy, header=galaxy_header, overwrite=True)

    # Add the simulated convolved galaxy image to the original science image
    final_image = galaxy_data + convolved_simulated

    # Save the final image with the simulated galaxy in the relevant folder
    final_output_path = os.path.join(simulation_dir, f'sci_convolved_{index}.fits')
    fits.writeto(final_output_path, final_image, header=galaxy_header, overwrite=True)



In [6]:
# Create a DataFrame from the lists of input parameters
input_params_df = pd.DataFrame({
    'Sersic_Index': sersic_index_list,
    'Effective_Radius': effective_radius_list,
    'Amplitude': amplitude_list,
    'Ellipticity': ellipticity_list,
    'Theta': theta_list,
    'Total_Flux': total_flux_list,
    'Magnitude': magnitude_list
})

# Set the index of the DataFrame to be the simulation number
input_params_df.index.name = 'Simulation_Number'

# Display the DataFrame
print(input_params_df)

# Save the DataFrame to a CSV file
input_params_df.to_csv('input_params.csv')

                   Sersic_Index  Effective_Radius  Amplitude  Ellipticity  \
Simulation_Number                                                           
0                      1.561810          4.901429   0.007588     0.419732   
1                      1.233992          3.116167   0.008796     0.420223   
2                      1.030877          4.939820   0.008492     0.342468   
3                      1.275107          3.608484   0.005723     0.386389   
4                      1.917779          3.278988   0.003629     0.373272   
...                         ...               ...        ...          ...   
115                    2.424281          3.294147   0.009339     0.398423   
116                    1.688704          4.960065   0.005434     0.365750   
117                    1.360218          3.151727   0.002160     0.325609   
118                    1.208241          4.281749   0.002637     0.369133   
119                    1.710942          4.335115   0.002551     0.338458   

In [7]:
# Assuming coordinates2 and input_params_df are already defined DataFrames
merged_df = pd.merge(coordinates, input_params_df, left_index=True, right_index=True)

# Display the merged DataFrame
print(merged_df)

# Save the DataFrame to a CSV file
merged_df.to_csv('merged_input_params.csv')

     Unnamed: 0   x0   y0  Sersic_Index  Effective_Radius  Amplitude  \
0             0  202  535      1.561810          4.901429   0.007588   
1             1  370  206      1.233992          3.116167   0.008796   
2             2  171  120      1.030877          4.939820   0.008492   
3             3  221  566      1.275107          3.608484   0.005723   
4             4  314  430      1.917779          3.278988   0.003629   
..          ...  ...  ...           ...               ...        ...   
115         115  599  610      2.424281          3.294147   0.009339   
116         116  400  231      1.688704          4.960065   0.005434   
117         117  514  544      1.360218          3.151727   0.002160   
118         118  190  630      1.208241          4.281749   0.002637   
119         119  138  225      1.710942          4.335115   0.002551   

     Ellipticity     Theta  Total_Flux  Magnitude  
0       0.419732  0.490147    1.590454  23.396197  
1       0.420223  2.224476    0

In [12]:
# Save the convolved simulated galaxy fits file in the relevant folder
simulation_dir = f'simulation_{index}'

# Define a function to crop data and save
def crop_data_and_save(input_filename, x_0, y_0, simulation_dir):
    # Load the original FITS data
    data, header = fits.getdata(input_filename, header=True)

    # Define the size of the cropped region
    crop_size = 400  # Assuming you want a 300x300 pixels crop

    # Calculate the starting and ending indices along each axis
    start_x = int(x_0 - crop_size / 2)
    end_x = int(x_0 + crop_size / 2)
    start_y = int(y_0 - crop_size / 2)
    end_y = int(y_0 + crop_size / 2)

    # Crop the data
    cropped_data = data[start_y:end_y, start_x:end_x]

    # Write the cropped data to a new FITS file
    fits.writeto(cropped_output_filename, cropped_data, header, overwrite=True)

# Process each simulation
for index, row in coordinates.iterrows():
    # Define the central coordinates for this simulation
    center_x = row['x0']
    center_y = row['y0']

    # Define the directory for this simulation
    simulation_dir = f'simulation_{index}'

    # Create the simulation directory if it doesn't exist
    if not os.path.exists(simulation_dir):
        os.makedirs(simulation_dir)

    # Define the input filename
    input_filename = f'simulation_{index}/sci_convolved_{index}.fits'
    cropped_output_filename = f'simulation_{index}/cropped_sci_convolved_{index}.fits'
   
    # Crop and save the data
    crop_data_and_save(input_filename, center_x, center_y, simulation_dir)


In [9]:
import pandas as pd

# Load the CSV file into a DataFrame
input_params_df = pd.read_csv('merged_input_params.csv')

# Find the maximum value of the 'Total_Flux' column
max_flux = input_params_df['Total_Flux'].max()

# Identify the simulations where the maximum flux occurs
max_flux_simulations = input_params_df[input_params_df['Total_Flux'] == max_flux]

# Find the index of the row with the maximum value of the 'Total_Flux' column
max_flux_row_index = input_params_df['Total_Flux'].idxmax()

# Retrieve the row with the maximum flux
max_flux_row = input_params_df.loc[max_flux_row_index]

print("Row with Maximum Flux:")
print(max_flux_row)

print("Maximum Flux Value:", max_flux)


Row with Maximum Flux:
Unnamed: 0.1         48.000000
Unnamed: 0           48.000000
x0                  596.000000
y0                  541.000000
Sersic_Index          2.410688
Effective_Radius      4.907857
Amplitude             0.009234
Ellipticity           0.374032
Theta                 0.048558
Total_Flux            3.127538
Magnitude            22.661994
Name: 48, dtype: float64
Maximum Flux Value: 3.127537722028495


In [10]:
import pandas as pd

# Load the CSV file into a DataFrame
input_params_df = pd.read_csv('merged_input_params.csv')

# Print the data from the first row to the tenth row
print(input_params_df.iloc[:11])

    Unnamed: 0.1  Unnamed: 0   x0   y0  Sersic_Index  Effective_Radius  \
0              0           0  202  535      1.561810          4.901429   
1              1           1  370  206      1.233992          3.116167   
2              2           2  171  120      1.030877          4.939820   
3              3           3  221  566      1.275107          3.608484   
4              4           4  314  430      1.917779          3.278988   
5              5           5  558  187      2.177764          3.399348   
6              6           6  472  199      1.911317          3.341048   
7              7           7  230  408      2.212596          3.609228   
8              8           8  443  591      1.183057          3.990354   
9              9           9  260  559      1.993783          3.623422   
10            10          10  352  660      2.454377          4.550266   

    Amplitude  Ellipticity     Theta  Total_Flux  Magnitude  
0    0.007588     0.419732  0.490147    1.590454 